In [ ]:
# LATEST VERSION 1.09

In [ ]:
import os
import time
import requests
import json
from flask import Flask, request, render_template_string

# --- Flask App Initialization ---
app = Flask(__name__)

# --- Configuration and Initialization ---
# Add Your Gemini Key Below
API_KEY = os.getenv('GEMINI_API_KEY', 'gemini-key-here')

# Ensure the model is correct as specified in the instructions
MODEL_NAME = "gemini-2.5-flash-preview-09-2025"
API_URL = f"https://generativelanguage.googleapis.com/v1beta/models/{MODEL_NAME}:generateContent?key={API_KEY}"

# Agent Persona: A dedicated Data Analysis and Insight Specialist.
SYSTEM_INSTRUCTION = (
    "You are a sophisticated Data Analysis and Insight Specialist, functioning as a Senior Data Analyst. "
    "Your output MUST be a comprehensive report based on the user's dataset description or analysis topic. "
    "The output MUST be structured using Markdown headings and lists to clearly include five key components: "
    "## Data Usability Score: [1-10]\n\n"
    "## Usability Assessment\n\n"
    "## Key Metrics & Suggested Calculations (Including YOY/Time Series)\n\n"
    "## Simulated EDA & Insights\n\n"
    "## Next Steps for a Senior Analyst\n\n"
    "Use Google Search for current, grounded information when discussing general data analysis topics or external datasets. "
    "Deliver the complete analysis directly without any preambles or conversational text."
)

# --- Core API Call Function with Retry Logic ---

def call_gemini_api_with_retry(payload):
    """Handles the API call to Gemini with exponential backoff for reliability."""
    max_retries = 5
    initial_delay = 1  # seconds

    if not API_KEY:
        return "⚠️ **ERROR:** Gemini API Key is missing. Please set the API_KEY variable in the script."

    for attempt in range(max_retries):
        try:
            headers = {
                'Content-Type': 'application/json'
            }
            response = requests.post(API_URL, headers=headers, data=json.dumps(payload))
            response.raise_for_status()

            result = response.json()
            
            if 'error' in result:
                 raise Exception(f"API returned error: {result['error']['message']}")

            text = result.get('candidates', [{}])[0].get('content', {}).get('parts', [{}])[0].get('text', 'No output generated.')
            return text

        except requests.exceptions.RequestException as e:
            if attempt < max_retries - 1:
                delay = initial_delay * (2 ** attempt)
                time.sleep(delay)
            else:
                return f"❌ **Error:** Failed to communicate with the Gemini API after {max_retries} attempts. Details: {e}"

        except Exception as e:
            return f"❌ **Error:** An unexpected error occurred: {e}"

# --- Agent Logic ---

def data_analysis_agent(task_prompt):
    """
    Executes a data analysis or EDA task using the Gemini API based on a description.
    """
    user_query = task_prompt

    payload = {
        "contents": [{ "parts": [{ "text": user_query }] }],
        "systemInstruction": { "parts": [{ "text": SYSTEM_INSTRUCTION }] },
        "tools": [{ "google_search": {} }], 
    }

    result_text = call_gemini_api_with_retry(payload)
    
    return result_text

# --- Flask  and HTML Template ---

HTML_TEMPLATE = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>InsightPilot Senior Data Analyst</title>
    <!-- Tailwind CSS CDN -->
    <script src="https://cdn.tailwindcss.com"></script>
    <!-- Showdown.js for Markdown Rendering -->
    <script src="https://cdnjs.cloudflare.com/ajax/libs/showdown/2.1.0/showdown.min.js"></script>
    <style>
        @import url('https://fonts.googleapis.com/css2?family=Inter:wght@400;600;700;800&display=swap');
        body { font-family: 'Inter', sans-serif; background-color: #f1f5f9; } /* Subtle light blue-gray background */
        
        /* Custom markdown styling for results */
        .markdown-content {
            padding: 1rem;
            color: #1f2937; /* Gray-800 */
        }
        .markdown-content h2 { 
            font-size: 1.75rem; /* Larger heading */
            font-weight: 800; /* Extra bold for headers */
            margin-top: 1.75rem; 
            margin-bottom: 0.75rem; 
            padding-bottom: 0.5rem; 
            border-bottom: 3px solid #3b82f6; /* Blue-500 */
            color: #1e40af; /* Darker Blue for emphasis */
        }
        .markdown-content p, .markdown-content ul, .markdown-content ol { 
            margin-bottom: 1rem; 
            line-height: 1.75; 
        }
        .markdown-content ul, .markdown-content ol { 
            list-style-type: disc; 
            padding-left: 1.5rem; 
        }
        .markdown-content li { 
            margin-bottom: 0.5rem; 
        }
    </style>
</head>
<body class="min-h-screen flex flex-col items-center bg-gray-50">

    <div class="w-full max-w-5xl bg-white rounded-xl shadow-2xl overflow-hidden border border-gray-100 mt-10 mb-10">

        <!-- Header Bar (Modern Gradient and Prominent) -->
        <header class="bg-gradient-to-r from-blue-700 to-indigo-600 text-white p-8 shadow-2xl">
            <div class="flex flex-col sm:flex-row items-start sm:items-center justify-between">
                <h1 class="text-4xl font-extrabold mb-1 sm:mb-0 flex items-center">
                    <span class="mr-3 text-5xl font-mono">📊</span> InsightPilot AI Agent
                </h1>
                <p class="text-lg opacity-90 mt-2 sm:mt-0 italic font-light">Advanced Usability, Metrics, and EDA Simulation</p>
            </div>
        </header>

        <!-- Main Content Area -->
        <div class="p-8 md:p-10 lg:p-12">
            
            <!-- Input Form (Upper Section - Highlighting the Input Area) -->
            <section class="mb-12 p-8 bg-gray-50 border border-gray-200 rounded-xl shadow-inner">
                <form id="analysis-form" method="post" class="space-y-6">
                    <label for="task_prompt" class="block text-xl font-semibold text-gray-800 mb-4">
                        Describe the dataset's columns, size, and analysis goals:
                    </label>
                    <textarea id="task_prompt" name="task_prompt" rows="6" 
                              class="w-full p-5 text-lg border-2 border-gray-300 rounded-xl focus:ring-indigo-500 focus:border-indigo-500 transition duration-200 shadow-md resize-none placeholder-gray-400"
                              placeholder="Example: 'Analyze a dataset on e-commerce sales with columns: transaction_id, customer_segment, purchase_value, purchase_date. I need a YOY sales breakdown and churn insights.'"></textarea>
                    
                    <div class="flex justify-end pt-4">
                        <button type="submit" 
                                class="w-full sm:w-80 px-8 py-4 bg-gradient-to-r from-blue-600 to-indigo-600 text-white font-bold rounded-xl shadow-xl hover:shadow-indigo-500/50 transition duration-300 transform hover:scale-[1.02] flex items-center justify-center space-x-2">
                            <span>Generate Senior Analysis</span>
                            <svg class="w-6 h-6" fill="none" stroke="currentColor" viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg"><path stroke-linecap="round" stroke-linejoin="round" stroke-width="2" d="M13 10V3L4 14h7v7l9-11h-7z"></path></svg>
                        </button>
                    </div>
                </form>
            </section>

            <!-- Result Display Area (Lower Section - Clean Report Look) -->
            <section id="result-section" class="mt-8">
                {% if result %}
                <div class="p-8 bg-white rounded-xl border-t-8 border-indigo-500 shadow-xl">
                    <h2 class="text-3xl font-bold text-indigo-800 mb-6 flex items-center border-none pb-0">
                        <svg class="w-8 h-8 mr-3 text-indigo-500" fill="none" stroke="currentColor" viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg"><path stroke-linecap="round" stroke-linejoin="round" stroke-width="2" d="M9 17v-2m3 2v-4m3 4v-6m2 10H7a2 2 0 01-2-2V5a2 2 0 012-2h5.586a1 1 0 01.707.293l5.414 5.414a1 1 0 01.293.707V19a2 2 0 01-2 2z"></path></svg>
                        Comprehensive Analysis Report
                    </h2>
                    
                    <!-- Hidden div to hold the raw markdown output -->
                    <div id="raw-markdown-output" style="display:none;">
                        {{ result }}
                    </div>
                    
                    <!-- Display div where rendered HTML will be placed -->
                    <div id="rendered-output" class="markdown-content">
                        <!-- Markdown will be rendered here by JS -->
                    </div>

                    <div class="mt-8 pt-4 border-t text-sm text-gray-500 italic text-center">
                        Note: This is a simulated, high-level analysis based on your textual description.
                    </div>
                </div>
                {% endif %}
            </section>
        </div>
    </div>

    <footer class="mt-12 text-sm text-gray-500">
        Powered by Gemini AI and Flask.
    </footer>

    <!-- Simple JS to handle loading state and Markdown rendering -->
    <script>
        document.getElementById('analysis-form').addEventListener('submit', function() {
            // Show Loading Overlay
            const overlay = document.createElement('div');
            overlay.innerHTML = `
                <div class="fixed inset-0 bg-black bg-opacity-40 flex items-center justify-center z-50">
                    <div class="bg-white p-8 rounded-xl shadow-2xl flex flex-col items-center space-y-4">
                        <svg class="animate-spin h-8 w-8 text-blue-600" xmlns="http://www.w3.org/2000/svg" fill="none" viewBox="0 0 24 24">
                            <circle class="opacity-25" cx="12" cy="12" r="10" stroke="currentColor" stroke-width="4"></circle>
                            <path class="opacity-75" fill="currentColor" d="M4 12a8 8 0 018-8V0C5.373 0 0 5.373 0 12h4zm2 5.291A7.962 7.962 0 014 12H0c0 3.042 1.135 5.824 3 7.938l3-2.647z"></path>
                        </svg>
                        <p class="text-xl font-medium text-gray-700">Running advanced data assessment...</p>
                        <p class="text-sm text-gray-500">Evaluating usability, calculating metrics, and simulating insights.</p>
                    </div>
                </div>`;
            document.body.appendChild(overlay.firstChild);
        });

        // Function to render Markdown
        window.onload = function() {
            const rawOutputDiv = document.getElementById('raw-markdown-output');
            const renderedOutputDiv = document.getElementById('rendered-output');

            if (rawOutputDiv && renderedOutputDiv) {
                const markdownText = rawOutputDiv.textContent.trim();
                
                // Initialize Showdown converter
                const converter = new showdown.Converter({
                    tables: true, 
                    simplifiedAutoLink: true,
                    strikethrough: true,
                    noHeaderId: true 
                });
                
                // Convert Markdown to HTML
                const html = converter.makeHtml(markdownText);
                
                // Inject rendered HTML into the display div
                renderedOutputDiv.innerHTML = html;
            }
        };
    </script>
</body>
</html>
"""

@app.route('/', methods=['GET', 'POST'])
def index():
    """Handles the main web page, input, and result display."""
    result = None
    
    if request.method == 'POST':
        task_prompt = request.form.get('task_prompt')
        if task_prompt:
            # Call the agent logic
            result = data_analysis_agent(task_prompt)
        else:
            result = "Please enter a dataset description or analysis task to proceed."

    return render_template_string(HTML_TEMPLATE, result=result)

if __name__ == '__main__':
    app.run(debug=True, use_reloader=False)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [18/Nov/2025 20:41:40] "GET / HTTP/1.1" 200 -
